In [1]:
import numpy as np
import torch
import random
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split

### Pytorch regression example
Just simple regression example using pytorch, mostly to shows how to use Dataset and Dataloader

1. Create samples
2. Dataset and splitting
3. Dataloader and NN-model
4. Train
5. Test

In [3]:
# Create samples
# Just adding two numbers together

X = []
Y = []
for _ in range(5000):
    x = [random.randint(0, 20), random.randint(0, 20)]
    y = [x[0] + x[1]]
    X.append(x)
    Y.append(y)

X = np.array(X)
Y = np.array(Y)

In [5]:
# Dataset and splitting
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=.33, random_state=26)

class Data(Dataset):
    def __init__(self, X, y):
        self.X = torch.from_numpy(X.astype(np.float32))
        self.y = torch.from_numpy(y.astype(np.float32))
        self.len = self.X.shape[0]

    def __getitem__(self, index):
        return self.X[index], self.y[index]

    def __len__(self):
        return self.len

In [6]:
# Dataloader and NN-model

# Instantiate training and test data
batch_size = 64
train_data = Data(X_train, Y_train)
train_dataloader = DataLoader(dataset=train_data, batch_size=batch_size, shuffle=True)

test_data = Data(X_test, Y_test)
test_dataloader = DataLoader(dataset=test_data, batch_size=batch_size, shuffle=True)


# Regression type neural network
class NeuralNetwork(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(NeuralNetwork, self).__init__()
        self.layer_1 = nn.Linear(input_dim, hidden_dim)
        nn.init.kaiming_uniform_(self.layer_1.weight, nonlinearity="relu")
        self.layer_2 = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        x = torch.nn.functional.relu(self.layer_1(x))
        x = self.layer_2(x)
        # x = torch.nn.functional.sigmoid(self.layer_2(x))
        return x

In [8]:
model = NeuralNetwork(2, 5, 1)
learning_rate = 0.01
loss_fn = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

num_epochs = 30
loss_values = []
loss = 0

for epoch in range(num_epochs):
    for X, y in train_dataloader:
        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        pred = model(X)

        # loss = loss_fn(pred.unsqueeze(-1), y.unsqueeze(-1))
        loss = loss_fn(pred, y)

        loss_values.append(loss.item())
        loss.backward()
        optimizer.step()
    if epoch % 5 == 0:
        print(loss.item())
        
print("Training Complete")

2.1009278297424316
0.003412672085687518
0.00037201523082330823
0.00021475469111464918
6.119583122199401e-05
2.3403636078001e-06
Training Complete


In [14]:
# Run through some tests
for _ in range(10):
    i1 = float(random.randint(0, 50))
    i2 = float(random.randint(0, 50))
    t = torch.tensor([i1, i2])
    predict = model(t)
    print(f"Addding {i1} and {i2} \t= {predict.item()}")

Addding 4.0 and 33.0 	= 37.005218505859375
Addding 22.0 and 41.0 	= 63.0075569152832
Addding 47.0 and 40.0 	= 87.00785827636719
Addding 9.0 and 44.0 	= 53.00800323486328
Addding 26.0 and 39.0 	= 65.00715637207031
Addding 31.0 and 7.0 	= 37.651588439941406
Addding 20.0 and 50.0 	= 70.00971984863281
Addding 25.0 and 27.0 	= 52.00420379638672
Addding 30.0 and 20.0 	= 50.00260925292969
Addding 50.0 and 1.0 	= 49.27699279785156
